In [3]:
import pickle

single_run = None
with open('pickles/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('pickles/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)

with open('pickles/single_run_pt.pkl', 'rb') as rf:
    single_run_pt = pickle.load(rf)

with open('pickles/n_run_pt.pkl', 'rb') as rf:
    n_runs_pt = pickle.load(rf)

with open('pickles/single_run_100g_pt.pkl', 'rb') as rf:
    single_run_100g_pt = pickle.load(rf)
    
with open('pickles/n_run_100g_pt.pkl', 'rb') as rf:
    n_run_100g_pt = pickle.load(rf)

In [2]:
import numpy as np
import time
from tabulate import tabulate
from evolution_utils import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score, all_models_score_table

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
c:\Users\student\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic 

In [5]:
chromo_set_1, score_set_1 = single_run_100g_pt

In [4]:
n_results = np.array(n_run_100g_pt)

C:\Users\student\AppData\Local\Temp\ipykernel_6708\1996161902.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  n_results = np.array(n_run_100g_pt)


In [6]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [7]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

In [8]:
def pick_top_n_genes_n_run(genes, chromo_set, n=1):
    common_index = dict()
    chromo_len_in_each_gen = []
    for chromo in chromo_set:
        if n==1:
            chromo_len_in_each_gen.append(np.count_nonzero(chromo == 1))
            for i in range(len(chromo)):
                if chromo[i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1
        else:
            best_gen = np.array(chromo[:, -1])
            chromo_len_in_each_gen.append(np.count_nonzero(best_gen[0] == 1))
            for i in range(len(best_gen[0])):
                if best_gen[0][i] == 1:
                    common_index[i] = common_index.setdefault(i, 0) + 1

    if genes == -1:
        genes = len(common_index)

    sorted_common_index = np.array(sorted(
        common_index.items(), key=lambda x: x[1], reverse=True)[:genes])
    return sorted_common_index[:, 0], sorted_common_index, np.array(chromo_len_in_each_gen)

# Collect Common indexes from res of n run evolution


# Final Results

In [9]:
def tabulate_results(chromo_set, n):
    genes = [30, 50, 60, 100, 150, 200, 250, -1]
    accuracy = []
    time_to_train = []
    sentiment_ratio = []
    for i in genes:
        top_n_genes, common_count, chromo_len = pick_top_n_genes_n_run(i, chromo_set, n)
        acc, ttt, sr = test_accuracy(top_n_genes)
        accuracy.append(acc)
        time_to_train.append(ttt)
        sentiment_ratio.append(sr)

    
    table_data = {'Genes': genes,
                'Accuracy': accuracy,
                'Time Taken to Train': time_to_train,
                'Sentiment Ratio(p/n)': sentiment_ratio
                }

    print('Actual chromosome length in X_Test: ', X_test.shape[1])
    print('RandomForest with full length -', all_models_score_table.iloc[0, 1:])
    print('Total Common genes count ~(-1): ', common_count)
    print(chromo_len)
    print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))
    return table_data, top_n_genes, common_count


Comparitive results for Single run evolution

In [10]:
single_run_table = tabulate_results(chromo_set_1, 1)
import pickle
with open('single_run_table.pkl', 'wb') as wf:
    pickle.dump(single_run_table, wf)
    wf.close()

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    0.890408
Name: 0, dtype: object
Total Common genes count ~(-1):  [[  35   99]
 [ 512   98]
 [ 599   98]
 ...
 [1439    1]
 [1460    1]
 [1474    1]]
[100 215 314 314 159 303 303 303 303 303 408 324 324 324 324 350 350 350
 352 352 352 352 352 352 424 424 424 424 436 487 425 425 406 410 410 410
 410 410 410 440 440 353 353 439 439 439 439 439 432 432 432 432 432 432
 487 422 437 435 435 435 442 442 491 449 449 449 449 449 449 449 449 449
 449 516 441 441 441 449 441 449 441 449 449 449 449 498 447 451 449 449
 445 445 445 445 445 445 445 445 445 445]
  Genes    Accuracy    Time Taken to Train  Sentiment Ratio(p/n)
-------  ----------  ---------------------  ----------------------
     30       0.612               0.171833  (18, 81)
     50       0.632               0.187454  (21, 78)
     60       0.664               0.203075  (24, 75)
    100       0.74                

Comparitive results for 30 run evolution

In [11]:
n_run_table, top_n_genes, common_count= tabulate_results(n_results, 30)
with open('n_run_table.pkl', 'wb') as wf:
    pickle.dump(n_run_table, wf)
    wf.close()

Actual chromosome length in X_Test:  1553
RandomForest with full length - Accuracy               0.8
Exec_Time_secs    0.890408
Name: 0, dtype: object
Total Common genes count ~(-1):  [[594  30]
 [599  30]
 [803  30]
 ...
 [ 70   1]
 [938   1]
 [448   1]]
[489 442 468 560 474 419 491 470 474 521 414 495 486 449 469 478 568 549
 515 454 445 473 566 400 453 502 439 533 523 556]
  Genes    Accuracy    Time Taken to Train  Sentiment Ratio(p/n)
-------  ----------  ---------------------  ----------------------
     30       0.84                0.187455  (43, 56)
     50       0.836               0.203076  (44, 55)
     60       0.84                0.203076  (45, 54)
    100       0.856               0.24994   (46, 53)
    150       0.876               0.296802  (49, 50)
    200       0.88                0.32998   (50, 50)
    250       0.872               0.359287  (49, 50)
     -1       0.8                 0.890409  (46, 54)


In [30]:
for i in common_count:
    print(i)

[594  30]
[599  30]
[461  27]
[803  26]
[1033   22]
[634  19]
[417  18]
[456  18]
[512  18]
[627  18]
[986  16]
[1527   16]
[1535   16]
[897  16]
[497  16]
[203  15]
[214  15]
[476  15]
[607  15]
[653  15]
[979  15]
[134  15]
[498  15]
[245  15]
[112  14]
[173  14]
[344  14]
[396  14]
[518  14]
[645  14]
[746  14]
[896  14]
[941  14]
[17 14]
[399  14]
[992  14]
[1009   14]
[307  14]
[308  14]
[323  14]
[445  14]
[715  14]
[330  14]
[92 13]
[119  13]
[251  13]
[268  13]
[349  13]
[372  13]
[381  13]
[440  13]
[530  13]
[569  13]
[611  13]
[616  13]
[633  13]
[704  13]
[810  13]
[813  13]
[890  13]
[1248   13]
[1479   13]
[1501   13]
[62 13]
[118  13]
[327  13]
[492  13]
[636  13]
[1007   13]
[1025   13]
[1037   13]
[1119   13]
[1446   13]
[223  13]
[243  13]
[671  13]
[956  13]
[967  13]
[69 13]
[187  13]
[216  13]
[1191   13]
[1303   13]
[23 12]
[162  12]
[166  12]
[170  12]
[179  12]
[209  12]
[215  12]
[238  12]
[290  12]
[304  12]
[424  12]
[438  12]
[509  12]
[540  12]
[541  12]
[5